<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/MN_T10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
#metodo de rayleigh

In [3]:
#metodo de iteracion en subespacio.
def gen_base_ortogonal(m,n):
    base=[]
    for j in range(m):
        b=[0]*n
        b[j]=1
        base.append(np.array(b))
    return np.array(base)
def factorizar_LU(matriz):#Codigo reciclado: Factorizacion LU
    n=len(matriz)
    L=np.zeros((n,n))
    U=np.zeros((n,n))
    for j in range(n):
        for i in range(j,n):
            sum=0
            for k in range(i):
                sum+=L[i][k]*U[k][j]
            L[i][j]=matriz[i][j]-sum
            if(i>j):
                sum=0
                for k in range(j):
                    sum+=L[j][k]*U[k][i]
                U[j][i]=(matriz[j][i]-sum)/L[j][j]
            else: 
                U[i][j]=1 
                if(L[i][i]==0):
                    print("Error en la construcción de L")
    return([L,U])

def superior(list,b):#Codigo reciclado:Resolver sistema triangular
  n=len(list)
  X=[b[n-1]/list[n-1][n-1]]
  X_inv=[X[0]]
  for i in range(1,n):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[j]*list[n-i-1][n-1-j]
    X.append((b[n-1-i]-acum)/list[n-1-i][n-1-i])
    X_inv.insert(0,X[i])
  return X_inv

#Codigo reciclado:Resolver sistema triangular
def inferior(list,b):#list debe contener las entradas de la matriz triangular.
#Será aceptable introducir un arreglo de arreglos de distintos tamaños, para solo introducir las entradas no cero.
  X=[b[0]/list[0][0]]
  for i in range(1,len(list)):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[j]*list[i][j]
    X.append((b[i]-acum)/list[i][i])
  return X
def solve(L_m,U_m,b):
    y=inferior(L_m,b)#Resolver Ly=b
    return(superior(U_m,y))#Resolver Ux=y
def check_diag(A,tol):#Pensado para matrices en general (NO simetricas)
    boolean=True
    breaker=False
    for i in range(len(A)):
        for j in range(i,len(A)):
            if(i!=j):
                if(abs(A[i][j])>tol):
                    boolean=False;
                    breaker=True;
                    break;
        if(breaker==True):
            break
    return(boolean)
def Jacobi_rot(A,tol,max_Iter):
    count=0
    n=len(A)
    P=np.identity(n)
    while((check_diag(A,tol)==False)&(count<=max_Iter)):
        p,q=entrada_max(A)
        a=A[p][q]
        if(A[p][p]==A[q][q]):
            theta=np.pi/4
        else:
            theta=0.5*np.arctan(2*a/(A[p][p]-A[q][q]))
        #Ya tenemos la rotacion y podemos hacer el producto de matrices
        R=np.identity(n)
        R[p][p]=np.cos(theta)
        R[p][q]=-np.sin(theta)
        R[q][p]=-R[p][q]
        R[q][q]=R[p][p]
        A=np.dot(R.T,np.dot(A,R))#Se puede mejorar, modificando solo aquellas entradas que cambian.
        count+=1
        P=np.dot(P,R)
    if(count>max_Iter):
        print("Maxima iteracion alcanzada")
    else:
        print("num. Iteraciones: ", str(count))
    return(P,np.diagonal(A))
def entrada_max(A):#Regresa una lista [i,j], tal que a_{i,j} es el valor maximo en valor absoluto.
#Pensado para matrices simetricas
    max=abs(A[0][1])
    index=[0,1]
    for i in range(len(A)):
        for j in range(i,len(A[0])):
            if(i!=j):
                if(max<abs(A[i][j])):
                    max=abs(A[i][j])
                    index=i,j
    return(index)

In [4]:
##Algoritmo
##1 Genera sub-base ortonormal
## Usa ese vector inicial (con deflación) para UNA iteración de potencia inversa (o potencia.) 
#-> eso genera una nueva matriz \Phi de approximaciones de vectores. Se normalizan.
## Hacer el producto B=\Phi.T*A*\Phi. Parar cuando esto sea diagonal.
## Aplicar Jacobi (Completo) 
#-> Genera un producto de matrices de rotacion Q, que puedo usar como aproximacion de eigenvectores \Phi.
##Repetir

In [5]:
U=gen_base_ortogonal(3,4)
print(U[1:1])
np.matrix(U)

[]


matrix([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0]])

In [6]:
def quitar_contribucion(v_0,list_eigen):
    if list_eigen==[]:
        print("no se quito nada")
        return(np.array(v_0))
    else:
        aux=v_0
        for v in list_eigen:
            v_0-=np.dot(aux,np.array(v))*np.array(v)
        print("se quitaron: ",list_eigen)
        return v_0


In [7]:
np.dot([[2,0],[0,2]],np.array([1,2]))

array([2, 4])

In [46]:
def Iter_subespacio(A,m,tol=0.000001,max_iter=1000):
    n=len(A)
    phi_old=[[1,0,0],[0,1,0]]
    B=A
    count=0;
    while(check_diag(B,tol=0.00001)==False & count<max_iter):
        count+=1
        print("phi_old:\n ",phi_old)
        for j in range(m):
            v_i=phi_old[j]
            if(j>0):
                aux=phi_old[j]
                contrib=[0,0,0]
                for k in range(j):
                    v_i-=np.dot(aux,phi_old[k])*np.array(phi_old[k])
            v_i_next=np.linalg.solve(A,v_i)
            phi_old[j]=np.array(v_i_next)/np.linalg.norm(v_i_next)
        B=np.dot(np.dot(phi_old,A),np.array(phi_old).T)
        print("B::::::::::",B)
        print("Jac",Jacobi_rot(B,tol,max_iter)[0])
        print("P.T",np.array(phi_old).T)
        phi_old=np.dot(np.array(phi_old).T,Jacobi_rot(B,tol,max_iter)[0]).T
    print("Num. iteraciones", count)
    return (np.dot(np.array(phi_old),phi_old.T))

In [9]:
U=[1,1,1]
type(U/np.linalg.norm(np.asarray(U)))

numpy.ndarray

In [47]:
Iter_subespacio([[4,-14,-12],[-14,10,13],[-12,13,1]],m=2)

phi_old:
  [[1, 0, 0], [0, 1, 0]]
B:::::::::: [[-6.76142750e+00 -1.30035450e-15]
 [-1.53835776e-15 -5.52933603e+00]]
num. Iteraciones:  0
Jac [[1. 0.]
 [0. 1.]]
P.T [[-7.16179888e-01 -3.18218036e-17]
 [-6.39607195e-01 -2.94062965e-01]
 [-2.79265114e-01  9.55786050e-01]]
num. Iteraciones:  0
Num. iteraciones 1


array([[ 1.        , -0.07883291],
       [-0.07883291,  1.        ]])

In [33]:
np.linalg.eig([[4,-14,-12],[-14,10,13],[-12,13,1]])

(array([31.53568967, -9.64366467, -6.89202501]),
 array([[-0.55567116,  0.59474589, -0.58095343],
        [ 0.6635882 , -0.10370353, -0.74087535],
        [ 0.50087949,  0.7971969 ,  0.3370413 ]]))